# Synchronous PPO with PyBullet Ant

In [1]:
import datetime,gym,os,pybullet_envs,time,os,psutil,ray
import numpy as np
import tensorflow as tf
from util import gpu_sess,suppress_tf_warning,suppress_gym_warning
np.set_printoptions(precision=2)
suppress_tf_warning() 
suppress_gym_warning()
print ("Ready.")

Ready.


### PPO helper functions (most of them are copied from spinning up)

In [2]:
import scipy.signal

def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

def statistics_scalar(x, with_min_and_max=False):
    """
    Get mean/std and optional min/max of scalar x 
    Args:
        x: An array containing samples of the scalar to produce statistics for.
        with_min_and_max (bool): If true, return min and max of x in 
            addition to mean and std.
    """
    x = np.array(x, dtype=np.float32)
    global_sum, global_n = np.sum(x), len(x)
    mean = global_sum / global_n
    global_sum_sq = np.sum((x - mean)**2)
    std = np.sqrt(global_sum_sq / global_n)  # compute global std
    if with_min_and_max:
        global_min = (np.min(x) if len(x) > 0 else np.inf)
        global_max = (np.max(x) if len(x) > 0 else -np.inf)
        return mean, std, global_min, global_max
    return mean, std

def discount_cumsum(x, discount):
    """
    Compute discounted cumulative sums of vectors.
    input: 
        vector x, [x0, x1, x2]
    output:
        [x0 + discount * x1 + discount^2 * x2,  
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """
    def __init__(self, odim, adim, size=5000, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(combined_shape(size, odim), dtype=np.float32)
        self.act_buf = np.zeros(combined_shape(size, adim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.
        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """
        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        return [self.obs_buf, self.act_buf, self.adv_buf, 
                self.ret_buf, self.logp_buf]
        
def create_ppo_model(env=None,hdims=[256,256],output_actv=None):
    """
    Create PPO Actor-Critic Model (compatible with Ray)
    """
    import tensorflow as tf # make it compatible with Ray actors
    from gym.spaces import Box, Discrete
    
    def mlp(x, hdims=[64,64], actv=tf.nn.relu, output_actv=None):
        for h in hdims[:-1]:
            x = tf.layers.dense(x, units=h, activation=actv)
        return tf.layers.dense(x, units=hdims[-1], activation=output_actv)
    
    def mlp_categorical_policy(o, a, hdims=[64,64], actv=tf.nn.relu, output_actv=None, action_space=None):
        adim = action_space.n
        logits = mlp(x=o, hdims=hdims+[adim], actv=actv, output_actv=None)
        logp_all = tf.nn.log_softmax(logits)
        pi = tf.squeeze(tf.multinomial(logits,1), axis=1)
        logp = tf.reduce_sum(tf.one_hot(a, depth=adim) * logp_all, axis=1)
        logp_pi = tf.reduce_sum(tf.one_hot(pi, depth=adim) * logp_all, axis=1)
        return pi, logp, logp_pi, pi
    
    def gaussian_likelihood(x, mu, log_std):
        EPS = 1e-8
        pre_sum = -0.5 * (((x-mu)/(tf.exp(log_std)+EPS))**2 + 2*log_std + np.log(2*np.pi))
        return tf.reduce_sum(pre_sum, axis=1)
    
    def mlp_gaussian_policy(o, a, hdims=[64,64], actv=tf.nn.relu, output_actv=None, action_space=None):
        adim = a.shape.as_list()[-1]
        mu = mlp(x=o, hdims=hdims+[adim], actv=actv, output_actv=output_actv)
        log_std = tf.get_variable(name='log_std', initializer=-0.5*np.ones(adim, dtype=np.float32))
        std = tf.exp(log_std)
        pi = mu + tf.random_normal(tf.shape(mu)) * std
        logp = gaussian_likelihood(a, mu, log_std)
        logp_pi = gaussian_likelihood(pi, mu, log_std)
        return pi, logp, logp_pi, mu # <= mu is added for the deterministic policy
    
    def mlp_actor_critic(o, a, hdims=[64,64], actv=tf.nn.relu, 
                     output_actv=None, policy=None, action_space=None):
        if policy is None and isinstance(action_space, Box):
            policy = mlp_gaussian_policy
        elif policy is None and isinstance(action_space, Discrete):
            policy = mlp_categorical_policy

        with tf.variable_scope('pi'):
            pi, logp, logp_pi, mu = policy(
                o=o, a=a, hdims=hdims, actv=actv, output_actv=output_actv, action_space=action_space)
        with tf.variable_scope('v'):
            v = tf.squeeze(mlp(x=o, hdims=hdims+[1], actv=actv, output_actv=None), axis=1)
        return pi, logp, logp_pi, v, mu
    
    def placeholder(dim=None):
        return tf.placeholder(dtype=tf.float32,shape=(None,dim) if dim else (None,))
    
    def placeholders(*args):
        """
        Usage: a_ph,b_ph,c_ph = placeholders(adim,bdim,None)
        """
        return [placeholder(dim) for dim in args]
    
    def get_vars(scope):
        return [x for x in tf.compat.v1.global_variables() if scope in x.name]
    
    # Have own session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    
    # Placeholders
    odim = env.observation_space.shape[0]
    adim = env.action_space.shape[0]
    o_ph,a_ph,adv_ph,ret_ph,logp_old_ph = placeholders(odim,adim,None,None,None)
    
    # Actor-critic model 
    ac_kwargs = dict()
    ac_kwargs['action_space'] = env.action_space
    actor_critic = mlp_actor_critic
    pi,logp,logp_pi,v,mu = actor_critic(o_ph, a_ph, **ac_kwargs)
    
    # Need all placeholders in *this* order later (to zip with data from buffer)
    all_phs = [o_ph, a_ph, adv_ph, ret_ph, logp_old_ph]
    
    # Every step, get: action, value, and logprob
    get_action_ops = [pi, v, logp_pi]
    
    # Get variables
    pi_vars,v_vars = get_vars('pi'),get_vars('v')
    
    # Accumulate model
    model = {'o_ph':o_ph,'a_ph':a_ph,'adv_ph':adv_ph,'ret_ph':ret_ph,'logp_old_ph':logp_old_ph,
             'pi':pi,'logp':logp,'logp_pi':logp_pi,'v':v,'mu':mu,
             'all_phs':all_phs,'get_action_ops':get_action_ops,'pi_vars':pi_vars,'v_vars':v_vars}
    return model,sess

def create_ppo_graph(model,clip_ratio=0.2,pi_lr=3e-4,vf_lr=1e-3,epsilon=1e-2):
    """
    Create PPO Graph
    """
    # PPO objectives
    ratio = tf.exp(model['logp'] - model['logp_old_ph']) # pi(a|s) / pi_old(a|s)
    min_adv = tf.where(model['adv_ph']>0,
                       (1+clip_ratio)*model['adv_ph'], (1-clip_ratio)*model['adv_ph'])
    pi_loss = -tf.reduce_mean(tf.minimum(ratio * model['adv_ph'], min_adv))
    v_loss = tf.reduce_mean((model['ret_ph'] - model['v'])**2)
    
    # Info (useful to watch during learning)
    approx_kl = tf.reduce_mean(model['logp_old_ph'] - model['logp']) # a sample estimate for KL-divergence
    approx_ent = tf.reduce_mean(-model['logp']) # a sample estimate for entropy
    clipped = tf.logical_or(ratio > (1+clip_ratio), ratio < (1-clip_ratio))
    clipfrac = tf.reduce_mean(tf.cast(clipped, tf.float32))
    
    # Optimizers
    pi_ent_loss = pi_loss - 0.01 * approx_ent # entropy-reg policy loss 
    train_pi = tf.train.AdamOptimizer(learning_rate=pi_lr,epsilon=epsilon).minimize(pi_ent_loss)
    train_v = tf.train.AdamOptimizer(learning_rate=vf_lr,epsilon=epsilon).minimize(v_loss)
    
    # Accumulate graph
    graph = {'pi_loss':pi_loss,'v_loss':v_loss,'approx_kl':approx_kl,'approx_ent':approx_ent,
             'clipfrac':clipfrac,'train_pi':train_pi,'train_v':train_v}
    return graph

def update_ppo(model,graph,sess,buf,train_pi_iters=100,train_v_iters=100,target_kl=0.01):
    """
    Update PPO
    """
    feeds = {k:v for k,v in zip(model['all_phs'], buf.get())}
    pi_l_old, v_l_old, ent = sess.run(
        [graph['pi_loss'],graph['v_loss'],graph['approx_ent']],feed_dict=feeds)
    # Training
    for i in range(train_pi_iters):
        _, kl = sess.run([graph['train_pi'],graph['approx_kl']],feed_dict=feeds)
        if kl > 1.5 * target_kl: # if kl exceeds threshold
            break
    for _ in range(train_v_iters):
        sess.run(graph['train_v'],feed_dict=feeds)
    # Log changes from update
    pi_l_new,v_l_new,kl,cf = sess.run(
        [graph['pi_loss'],graph['v_loss'],graph['approx_kl'],graph['clipfrac']],
        feed_dict=feeds)
    
def save_ppo_model(npz_path,R,VERBOSE=True):
    """
    Save PPO model weights
    """
    # PPO model
    tf_vars = R.model['pi_vars'] + R.model['v_vars']
    data2save,var_names,var_vals = dict(),[],[]
    for v_idx,tf_var in enumerate(tf_vars):
        var_name,var_val = tf_var.name,R.sess.run(tf_var)
        var_names.append(var_name)
        var_vals.append(var_val)
        data2save[var_name] = var_val
        if VERBOSE:
            print ("[%02d]  var_name:[%s]  var_shape:%s"%
                (v_idx,var_name,var_val.shape,)) 
    
    # Create folder if not exist
    dir_name = os.path.dirname(npz_path)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print ("[%s] created."%(dir_name))
    # Save npz
    np.savez(npz_path,**data2save)
    print ("[%s] saved."%(npz_path))
    
def restore_ppo_model(npz_path,R,VERBOSE=True):
    """
    Restore PPO model weights
    """
    # Load npz
    l = np.load(npz_path)
    print ("[%s] loaded."%(npz_path))
    
    # Get values of PPO model  
    tf_vars = R.model['pi_vars'] + R.model['v_vars']
    var_vals = []
    for tf_var in tf_vars:
        var_vals.append(l[tf_var.name])       
    # Assign weights of PPO model
    R.set_weights(var_vals)
    
print ("Ready.")

Ready.


### Environment

In [3]:
def get_env(RENDER=False):
    import pybullet_envs,gym
    gym.logger.set_level(40) # gym logger
    eval_env = gym.make('AntBulletEnv-v0')
    if RENDER:
        _ = eval_env.render(mode='human') # enable rendering
        _ = eval_env.reset()
        for _ in range(3): # dummy run for proper rendering 
            a = eval_env.action_space.sample()
            o,r,d,_ = eval_env.step(a)
            time.sleep(0.01)
    return eval_env

print ("Ready.")

Ready.


### Rollout worker

In [4]:
class RolloutWorkerClass(object):
    """
    Worker without RAY (for update purposes)
    """
    def __init__(self,seed=1):
        self.seed = seed
        # Each worker should maintain its own environment
        import pybullet_envs,gym
        from util import suppress_tf_warning
        suppress_tf_warning() # suppress TF warnings
        gym.logger.set_level(40) # gym logger 
        self.env = get_env()
        odim,adim = self.env.observation_space.shape[0],self.env.action_space.shape[0]
        self.odim = odim
        self.adim = adim
        # Initialize PPO
        self.model,self.sess = create_ppo_model(env=self.env,hdims=hdims,output_actv=tf.nn.tanh)
        self.graph = create_ppo_graph(self.model,
                                      clip_ratio=clip_ratio,pi_lr=pi_lr,vf_lr=vf_lr,epsilon=epsilon)
        # Initialize model 
        tf.set_random_seed(self.seed)
        np.random.seed(self.seed)
        self.sess.run(tf.global_variables_initializer())
        
        # Flag to initialize assign operations for 'set_weights()'
        self.FIRST_SET_FLAG = True
        
    def get_action(self,o,deterministic=False):
        act_op = self.model['mu'] if deterministic else self.model['pi']
        return self.sess.run(act_op, feed_dict={self.model['o_ph']:o.reshape(1,-1)})[0]
    
    def get_weights(self):
        """
        Get weights
        """
        weight_vals = self.sess.run(self.model['pi_vars']+self.model['v_vars'])
        return weight_vals
    
    def set_weights(self,weight_vals):
        """
        Set weights without memory leakage
        """
        if self.FIRST_SET_FLAG:
            self.FIRST_SET_FLAG = False
            self.assign_placeholders = []
            self.assign_ops = []
            for w_idx,weight_tf_var in enumerate(self.model['pi_vars']+self.model['v_vars']):
                a = weight_tf_var
                assign_placeholder = tf.placeholder(a.dtype, shape=a.get_shape())
                assign_op = a.assign(assign_placeholder)
                self.assign_placeholders.append(assign_placeholder)
                self.assign_ops.append(assign_op)
        for w_idx,weight_tf_var in enumerate(self.model['pi_vars']+self.model['v_vars']):
            self.sess.run(self.assign_ops[w_idx],
                          {self.assign_placeholders[w_idx]:weight_vals[w_idx]})    
    
# Distributed workers    
@ray.remote
class RayRolloutWorkerClass(object):
    """
    Rollout Worker with RAY
    """
    def __init__(self,worker_id=0,ep_len_rollout=1000):
        # Parse
        self.worker_id = worker_id
        self.ep_len_rollout = ep_len_rollout
        # Each worker should maintain its own environment
        import pybullet_envs,gym
        from util import suppress_tf_warning
        suppress_tf_warning() # suppress TF warnings
        gym.logger.set_level(40) # gym logger 
        self.env = get_env()
        odim,adim = self.env.observation_space.shape[0],self.env.action_space.shape[0]
        self.odim = odim
        self.adim = adim
        # Replay buffers to pass
        self.o_buffer = np.zeros((self.ep_len_rollout,self.odim))
        self.a_buffer = np.zeros((self.ep_len_rollout,self.adim))
        self.r_buffer = np.zeros((self.ep_len_rollout))
        self.v_t_buffer = np.zeros((self.ep_len_rollout))
        self.logp_t_buffer = np.zeros((self.ep_len_rollout))
        # Create PPO model
        self.model,self.sess = create_ppo_model(env=self.env,hdims=hdims,output_actv=tf.nn.tanh)
        # Initialize model 
        self.sess.run(tf.global_variables_initializer())
        # Buffer
        self.buf = PPOBuffer(odim=self.odim,adim=self.adim,
                             size=ep_len_rollout,gamma=gamma,lam=lam)
        
        # Flag to initialize assign operations for 'set_weights()'
        self.FIRST_SET_FLAG = True
        
        # Flag to initialize rollout
        self.FIRST_ROLLOUT_FLAG = True
        
        print ("Worker [%d] initialized."%(self.worker_id))
        
    def get_action(self,o,deterministic=False):
        act_op = self.model['mu'] if deterministic else self.model['pi']
        return self.sess.run(act_op, feed_dict={self.model['o_ph']:o.reshape(1,-1)})[0]
    
    def set_weights(self,weight_vals):
        """
        Set weights without memory leakage
        """
        if self.FIRST_SET_FLAG:
            self.FIRST_SET_FLAG = False
            self.assign_placeholders = []
            self.assign_ops = []
            for w_idx,weight_tf_var in enumerate(self.model['pi_vars']+self.model['v_vars']):
                a = weight_tf_var
                assign_placeholder = tf.placeholder(a.dtype, shape=a.get_shape())
                assign_op = a.assign(assign_placeholder)
                self.assign_placeholders.append(assign_placeholder)
                self.assign_ops.append(assign_op)
        for w_idx,weight_tf_var in enumerate(self.model['pi_vars']+self.model['v_vars']):
            self.sess.run(self.assign_ops[w_idx],
                          {self.assign_placeholders[w_idx]:weight_vals[w_idx]})    
        
    def rollout(self):
        """
        Rollout
        """
        if self.FIRST_ROLLOUT_FLAG:
            self.FIRST_ROLLOUT_FLAG = False
            self.o = self.env.reset() # reset environment
        # Loop
        for t in range(ep_len_rollout):
            a,v_t,logp_t = self.sess.run(
                self.model['get_action_ops'],feed_dict={self.model['o_ph']:self.o.reshape(1,-1)})
            o2, r, d, _ = self.env.step(a[0])
            # save and log
            self.buf.store(self.o,a,r,v_t,logp_t)
            # Update obs (critical!)
            self.o = o2
            if d:
                self.buf.finish_path(last_val=0.0)
                self.o = self.env.reset() # reset when done 
        
        last_val = self.sess.run(self.model['v'],
                                 feed_dict={self.model['o_ph']:self.o.reshape(1,-1)})
        self.buf.finish_path(last_val)
        return self.buf.get()
    
print ("Ready.")

Ready.


### Hyperparameters

In [5]:
# Model
hdims = [256,256]

# Graph
clip_ratio = 0.2
pi_lr = 3e-4
vf_lr = 1e-3
epsilon = 1e-2

# Buffer
gamma = 0.99
lam = 0.95

# Update
train_pi_iters = 100
train_v_iters = 100
target_kl = 0.01
epochs = 1000
max_ep_len = 1000

# Worker 
n_cpu = n_workers = 5
total_steps,evaluate_every,print_every = 1000,50,10
ep_len_rollout = 1000
batch_size = 4096

print ("Ready.")

Ready.


### Initialize env

In [6]:
RENDER = False
eval_env = get_env(RENDER=RENDER)
adim,odim = eval_env.action_space.shape[0],eval_env.observation_space.shape[0]
print ("Environment Ready. odim:[%d] adim:[%d]."%(odim,adim))

Environment Ready. odim:[28] adim:[8].


### Initialize workers

In [7]:
ray.init(num_cpus=n_cpu)
tf.reset_default_graph()
R = RolloutWorkerClass(seed=0)
workers = [RayRolloutWorkerClass.remote(worker_id=i,ep_len_rollout=ep_len_rollout) 
           for i in range(n_workers)]
print ("RAY initialized with [%d] cpus and [%d] workers."%
       (n_cpu,n_workers))

2020-08-23 11:39:11,454	INFO resource_spec.py:231 -- Starting Ray with 37.01 GiB memory available for workers and up to 18.51 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-23 11:39:11,920	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


RAY initialized with [5] cpus and [5] workers.


### Loop

In [8]:
start_time = time.time()
n_env_step = 0 # number of environment steps
for t in range(int(total_steps)):
    esec = time.time()-start_time
    
    # 1. Synchronize worker weights
    weights = R.get_weights()
    set_weights_list = [worker.set_weights.remote(weights) for worker in workers] 
    
    # 2. Make rollout and accumulate to Buffers
    t_start = time.time()
    ops = [worker.rollout.remote() for worker in workers]
    rollout_vals = ray.get(ops)
    sec_rollout = time.time() - t_start
    
    # 3. Update
    t_start = time.time() # tic
    # Mini-batch type of update
    for r_idx,rval in enumerate(rollout_vals):
        obs_buf,act_buf,adv_buf,ret_buf,logp_buf = \
            rval[0],rval[1],rval[2],rval[3],rval[4]
        if r_idx == 0:
            obs_bufs,act_bufs,adv_bufs,ret_bufs,logp_bufs = \
                obs_buf,act_buf,adv_buf,ret_buf,logp_buf
        else:
            obs_bufs = np.concatenate((obs_bufs,obs_buf),axis=0)
            act_bufs = np.concatenate((act_bufs,act_buf),axis=0)
            adv_bufs = np.concatenate((adv_bufs,adv_buf),axis=0)
            ret_bufs = np.concatenate((ret_bufs,ret_buf),axis=0)
            logp_bufs = np.concatenate((logp_bufs,logp_buf),axis=0)
    n_val_total = obs_bufs.shape[0]
    for pi_iter in range(train_pi_iters):
        rand_idx = np.random.permutation(n_val_total)[:batch_size]
        buf_batches = [obs_bufs[rand_idx],act_bufs[rand_idx],adv_bufs[rand_idx],
                       ret_bufs[rand_idx],logp_bufs[rand_idx]]
        feeds = {k:v for k,v in zip(R.model['all_phs'],buf_batches)}
        _,kl,pi_loss,ent = R.sess.run([R.graph['train_pi'],R.graph['approx_kl'],
                               R.graph['pi_loss'],R.graph['approx_ent']],
                           feed_dict=feeds)        
        if kl > 1.5 * target_kl:
            # print ("  pi_iter:[%d] kl(%.3f) is higher than 1.5x(%.3f)"%(pi_iter,kl,target_kl))
            break
    for _ in range(train_v_iters):
        rand_idx = np.random.permutation(n_val_total)[:batch_size]
        buf_batches = [obs_bufs[rand_idx],act_bufs[rand_idx],adv_bufs[rand_idx],
                       ret_bufs[rand_idx],logp_bufs[rand_idx]]
        feeds = {k:v for k,v in zip(R.model['all_phs'],buf_batches)}
        R.sess.run(R.graph['train_v'],feed_dict=feeds)
    sec_update = time.time() - t_start # toc
    
    # Print
    if (t == 0) or (((t+1)%print_every) == 0): 
        print ("[%d/%d] pi_iter:[%d/%d] kl:[%.3f] target_kl:[%.3f] pi_loss:[%.3f] entropy:[%.2f]."%
               (t+1,total_steps,pi_iter,train_pi_iters,kl,target_kl,pi_loss,ent))
        print (" sec_rollout:[%.2f] sec_update:[%.2f]."%(sec_rollout,sec_update))
        
    # Evaluate
    if (t==0) or (((t+1)%evaluate_every) == 0):
        ram_percent = psutil.virtual_memory().percent # memory usage
        print ("[Eval. start] step:[%d/%d][%.1f%%] #step:[%.1e] time:[%s] ram:[%.1f%%]."%
               (t+1,total_steps,t/total_steps*100,
                n_env_step,
                time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time)),
                ram_percent)
              )
        o,d,ep_ret,ep_len = eval_env.reset(),False,0,0
        if RENDER:
            _ = eval_env.render(mode='human') 
        while not(d or (ep_len == max_ep_len)):
            a = R.sess.run(R.model['mu'],feed_dict={R.model['o_ph']:o.reshape(1,-1)})
            o,r,d,_ = eval_env.step(a[0])
            if RENDER:
                _ = eval_env.render(mode='human') 
            ep_ret += r # compute return 
            ep_len += 1
        print ("[Evaluate] ep_ret:[%.4f] ep_len:[%d]"%(ep_ret,ep_len))
        
        # Save 
        npz_path = '../net/ppo_ant/model.npz'
        save_ppo_model(npz_path,R,VERBOSE=False)

print ("Done.")

(pid=26958) Worker [3] initialized.
(pid=26954) Worker [1] initialized.
(pid=26957) Worker [4] initialized.
(pid=26955) Worker [0] initialized.
(pid=26956) Worker [2] initialized.
[1/1000] pi_iter:[99/100] kl:[0.007] target_kl:[0.010] pi_loss:[-0.008] entropy:[7.31].
 sec_rollout:[3.37] sec_update:[0.53].
[Eval. start] step:[1/1000][0.0%] #step:[0.0e+00] time:[00:00:03] ram:[5.4%].
[Evaluate] ep_ret:[429.5945] ep_len:[1000]
[../net/ppo_ant/model.npz] saved.
[10/1000] pi_iter:[99/100] kl:[0.008] target_kl:[0.010] pi_loss:[-0.011] entropy:[7.29].
 sec_rollout:[1.81] sec_update:[0.35].
[20/1000] pi_iter:[99/100] kl:[0.009] target_kl:[0.010] pi_loss:[-0.017] entropy:[7.12].
 sec_rollout:[1.81] sec_update:[0.29].
[30/1000] pi_iter:[99/100] kl:[0.009] target_kl:[0.010] pi_loss:[-0.018] entropy:[6.93].
 sec_rollout:[1.80] sec_update:[0.29].
[40/1000] pi_iter:[99/100] kl:[0.012] target_kl:[0.010] pi_loss:[-0.021] entropy:[6.72].
 sec_rollout:[1.79] sec_update:[0.33].
[50/1000] pi_iter:[99/100]

[500/1000] pi_iter:[15/100] kl:[0.015] target_kl:[0.010] pi_loss:[-0.001] entropy:[3.36].
 sec_rollout:[1.76] sec_update:[0.14].
[Eval. start] step:[500/1000][49.9%] #step:[0.0e+00] time:[00:16:59] ram:[5.5%].
[Evaluate] ep_ret:[2105.5995] ep_len:[1000]
[../net/ppo_ant/model.npz] saved.
[510/1000] pi_iter:[6/100] kl:[0.016] target_kl:[0.010] pi_loss:[-0.009] entropy:[3.36].
 sec_rollout:[1.79] sec_update:[0.13].
[520/1000] pi_iter:[5/100] kl:[0.016] target_kl:[0.010] pi_loss:[-0.001] entropy:[3.30].
 sec_rollout:[1.78] sec_update:[0.15].
[530/1000] pi_iter:[47/100] kl:[0.016] target_kl:[0.010] pi_loss:[-0.023] entropy:[3.14].
 sec_rollout:[1.76] sec_update:[0.24].
[540/1000] pi_iter:[6/100] kl:[0.016] target_kl:[0.010] pi_loss:[-0.009] entropy:[3.07].
 sec_rollout:[1.75] sec_update:[0.13].
[550/1000] pi_iter:[5/100] kl:[0.016] target_kl:[0.010] pi_loss:[0.001] entropy:[3.12].
 sec_rollout:[1.77] sec_update:[0.13].
[Eval. start] step:[550/1000][54.9%] #step:[0.0e+00] time:[00:18:39] ram

[Evaluate] ep_ret:[2532.0488] ep_len:[1000]
[../net/ppo_ant/model.npz] saved.
Done.


### Close

In [9]:
eval_env.close()
ray.shutdown()
print ("Env and ray closed.")

Env and ray closed.


### Restore

In [10]:
R.sess.run(tf.global_variables_initializer())
print ("Network initialized.")

Network initialized.


In [11]:
npz_path = '../net/ppo_ant/model.npz'
restore_ppo_model(npz_path,R,VERBOSE=False)

[../net/ppo_ant/model.npz] loaded.


### Evaluate

In [12]:
RENDER = False
eval_env = get_env(RENDER=RENDER)
o,d,ep_ret,ep_len = eval_env.reset(),False,0,0
if RENDER:
    _ = eval_env.render(mode='human') 
while not(d or (ep_len == max_ep_len)):
    a = R.sess.run(R.model['mu'],feed_dict={R.model['o_ph']:o.reshape(1,-1)})
    o,r,d,_ = eval_env.step(a[0])
    if RENDER:
        _ = eval_env.render(mode='human') 
    ep_ret += r # compute return 
    ep_len += 1
print ("[Evaluate] ep_ret:[%.4f] ep_len:[%d]"%(ep_ret,ep_len))
eval_env.close()

[Evaluate] ep_ret:[2573.5052] ep_len:[1000]
